In [2]:
import os
os.environ['MPLCONFIGDIR'] = "/home/mventura/latent_ode_expts"
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches
%load_ext autoreload 
%autoreload 2
%matplotlib inline

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-4txcz0sr because the default path (/home/mventura/latent_ode_expts) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Consider an early

In [74]:
!python3 run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic_rm --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic_rm --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 55455
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 1348.913330 | Likelihood -1375.684937 | KL fp 6.0831 | FP STD 1.3000|
KL coef: 0.0
Train loss (one batch): 1473.3492431640625
Train CE loss (one batch): 0.0
Test MSE: 0.2759
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 55455
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 1175.322021 | Likelihood -1184.516479 | KL fp 5.3766 | FP STD 0.8240|
KL coef: 0.0
Train loss (one batch): 1210.1583251953125
Train CE loss (one batch): 0.0
Test MSE: 0.2376
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 55455
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 1139.493530 | Likelihood -1142.809204 | KL fp 5.6954 | FP STD 0.7224|
KL coef: 0.0
Train loss (o

plotting....
Computing loss... 0
Experiment 55455
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 872.155762 | Likelihood -872.437561 | KL fp 5.6390 | FP STD 0.1514|
KL coef: 0.15705680661607324
Train loss (one batch): 939.6063232421875
Train CE loss (one batch): 0.0
Test MSE: 0.1752
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 55455
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 870.652161 | Likelihood -870.920593 | KL fp 5.0204 | FP STD 0.2043|
KL coef: 0.1654862385499125
Train loss (one batch): 939.6981201171875
Train CE loss (one batch): 0.0
Test MSE: 0.1749
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 55455
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 863.413025 | Likelihood -863.629944 | KL fp 5.2166 | FP STD 0.1562|
KL coef: 0.17383137616441335
Train loss (one batch): 932.3134155273438
Train CE loss (one batch): 0.0
Test MSE: 0.1735
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss..

In [75]:
!python3 run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic_rm --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic_rm --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 26828
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 1313.032471 | Likelihood -1317.010742 | KL fp 5.5648 | FP STD 0.5527|
KL coef: 0.0
Train loss (one batch): 1258.0255126953125
Train CE loss (one batch): 0.0
Test MSE: 0.2641
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 26828
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 1159.322021 | Likelihood -1161.687866 | KL fp 6.5318 | FP STD 0.5432|
KL coef: 0.0
Train loss (one batch): 1210.232421875
Train CE loss (one batch): 0.0
Test MSE: 0.2331
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 26828
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 1143.391602 | Likelihood -1152.099731 | KL fp 6.7876 | FP STD 0.6648|
KL coef: 0.0
Train loss (one ba

plotting....
Computing loss... 0
Experiment 26828
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 914.453125 | Likelihood -914.356018 | KL fp 8.2425 | FP STD 0.2796|
KL coef: 0.15705680661607324
Train loss (one batch): 976.4209594726562
Train CE loss (one batch): 0.0
Test MSE: 0.1836
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 26828
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 913.662048 | Likelihood -913.514343 | KL fp 8.1311 | FP STD 0.2657|
KL coef: 0.1654862385499125
Train loss (one batch): 976.9658203125
Train CE loss (one batch): 0.0
Test MSE: 0.1834
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 26828
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 913.424377 | Likelihood -913.203857 | KL fp 8.1108 | FP STD 0.2506|
KL coef: 0.17383137616441335
Train loss (one batch): 976.0529174804688
Train CE loss (one batch): 0.0
Test MSE: 0.1834
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0